<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality and Cleanliness

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Dataset:** Homicides Colombia ([datos.gov.co](datos.gov.co))

**References:**
- String similarity metrics: https://www.baeldung.com/cs/string-similarity-edit-distance
- Levenshtein lib's docs: https://rapidfuzz.github.io/Levenshtein/installation.html
  
Last update: September, 2024

In [ ]:
!pip install levenshtein

In [1]:
import re
from datetime import datetime

import numpy as np
import pandas as pd

from Levenshtein import distance, ratio

In [2]:
pd.set_option("display.max_columns", None)

### Load the data

In [3]:
homicides_df = pd.read_csv("./data/homicides.csv")

In [4]:
homicides_df.shape

(12400, 22)

In [5]:
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [6]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
11670,12/13/2021 12:00:00 AM,VALLE,CALI (CT),Domingo,3:14,CHARCO AZUL E13,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,51.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1964.0,77-802,taebpa1058@unidatos.edu.co
9603,10/15/2021 12:00:00 AM,NARIÑO,SAN ANDRES DE TUMACO,juves,18:05,EL VOLADERO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,CONDUCTOR TAXI,A PIE,34.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,52835000.0,1981.0,92-476,egqsnf5353@unidatos.edu.co
2065,03/05/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,9:00,VILLA DEL SOCORRO CNO REPORTADO2,URBANA,CASAS DE HABITACION,CONTUNDENTES,A PIE,A PIE,35.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1980.0,56-301,qqrhuk1408@unidatos.edu.co
7179,07/31/2021 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Viernes,23:40,LUCERO ALTO ENO REPORTADO19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1988.0,89-570,dfjurl9333@gmail.com
3332,04/12/2021 12:00:00 AM,ANTIOQUIA,FRONTINO,Domingo,14:00,MURINDO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,32.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5284000.0,1983.0,15-049,kjpkqt2743@unidatos.edu.co


### Work with datetimes

In [7]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [11]:
# Applying the validation to all values in the column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR BECAUSE SOME VALUES DOESN'T FIT THE FORMAT

ValueError: time data '13/12/2021 12:00:00 AM' does not match format '%m/%d/%Y %H:%M:%S %p'

In [9]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [10]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
486,13/12/2021 12:00:00 AM,VALLE,CALI (CT),kunes,23:00,MOJICA E15,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,89.0,80-330,lujhdf9132@gmail.com
695,30/01/2021 12:00:00 AM,BOLÍVAR,CARTAGENA (CT),Lunes,5:30,REP. DEL LIBANO,URBANA,VIAS PUBLICAS,ARMA BLANCA,NO REPORTADO,A PIE,25.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,13001000.0,1990.0,12-915,ghumtg4094@unidatos.edu.co
1250,18/05/2021 12:00:00 AM,HUILA,TESALIA,Jueves,19:30,VEREDA PACARNI,RURAL,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,34.0,FEMENINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,41797000.0,1981.0,99-095,sdaggf6639@gmail.com
12168,12/25/2021 12:00:00 MM,VALLE,PALMIRA,Viernes,15:30,LA EMILIA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,17.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1998.0,16-362,uschca1775@gmail.com
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Deleting a row by its index
homicides_df.drop([486, 695, 1250, 12168, 12399], inplace=True)

In [13]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [14]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
HORA                         object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [15]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

FECHA
12    12395
Name: count, dtype: int64

<span style="color:red">The homicide hour is available in a different column!</span>

In [16]:
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11]) + homicides_df["HORA"]

In [17]:
# Deleting redundant column
homicides_df.drop(columns=["HORA"], inplace=True)

In [18]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [19]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [20]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [21]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
6250,2021-07-04 10:45:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Sábado,NACIONES UNIDAS ENO REPORTADO19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,31.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1984.0,68-879,dgppnd5070@unidatos.edu.co
2852,2021-03-28 08:09:00,CUNDINAMARCA,BOGOTÁ D.C. (CT),Sabadi,SANTA ISABEL ENO REPORTADO14,URBANA,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,50.0,MASCULINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,SUPERIOR,COLOMBIA,11001000.0,1965.0,52-537,cllnnd2811@gmail.com
7842,2021-08-21 19:50:00,META,GRANADA,Viernes,VILLAS DE GRANADA,URBANA,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,35.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,50313000.0,1980.0,17-939,correo439@unidatos.edu.co
8353,2021-09-06 02:00:00,CAQUETÁ,FLORENCIA (CT),Domingo,JUAN XXIII,URBANA,HOSPITALES,ARMA DE FUEGO,A PIE,A PIE,30.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1985.0,96-957,hahhpk1003@unidatos.edu.co
2051,2021-03-04 22:40:00,SANTANDER,BARRANCABERMEJA,Miércoles,SAN MARTIN,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,PASAJERO MOTOCICLETA,18.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,68081000.0,1997.0,71-054,mcahtg2668@gmail.com


In [22]:
homicides_df["FECHA"].min(), homicides_df["FECHA"].max()

(Timestamp('2021-01-01 00:00:00'), Timestamp('2021-12-31 23:58:00'))

### Fix categorical column

In [23]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [24]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary.</span>

<span style="color:red">Hint: You can use the replace() pandas function.</span>

### Analyze potential duplicates

In [25]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [26]:
duplicates_by_cedula.shape

(1627, 21)

In [27]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated.</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

### Fix formats

In [28]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [29]:
cedula_malformed.shape

(28, 21)

In [30]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,188-03,diebuo5651@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

<span style="color:red">Assume that the third digit in the first part corresponds to the first one in the second part.</span>

<span style="color:red">TODO:Check the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

### Automate imputation of categorical values

In [ ]:
homicides_df["DIA"].unique()

In [ ]:
# Calculating the distance between two words using the Levenshtein distance
print(distance("sábado", "sabaod"))
print(distance("sábado", "viernes"))

In [ ]:
print(ratio("sábado", "sabaod"))
print(ratio("sábado", "viernes"))

<span style="color:red">TODO: Create a function to automatically fix the digitation errors for the "DIA" column.</span>

In [ ]:
# It's expected to hava a list with only 7 values
homicides_df["DIA"].unique()

### Recalculate based on a different column

In [ ]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

<span style="color:red">TODO: Fix "EDAD" column using "AÑO DE NACIMIENTO".</span>

### Conclusion

<span style="color:red">Make a summary of the different data quality issues found on the dataset, the data quality dimension that is related to and the implemented strategy for solving or mitigating each issue.</span>